# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Z.-L. Xie  ->  Z.-L. Xie  |  ['Z.-L. Xie']
N. Winkel  ->  N. Winkel  |  ['N. Winkel']
I. Smirnova-Pinchukova  ->  I. Smirnova-Pinchukova  |  ['I. Smirnova-Pinchukova']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Li  ->  J. Li  |  ['J. Li']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
Arxiv has 62 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.16757


extracting tarball to tmp_2309.16757...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

/tmp/ipykernel_2245/3009462391.py:49: LatexWarning: 2309.17287 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2309.17440


extracting tarball to tmp_2309.17440...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.17440/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.16757-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.16757) | **A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE):  JWST Discovers an Overdensity around a Metal Absorption-selected Galaxy at  $z\sim5.5$**  |
|| Y. Wu, et al. -- incl., <mark>E. Bañados</mark>, <mark>Z.-L. Xie</mark> |
|*Appeared on*| *2023-10-02*|
|*Comments*| *Accepted for publication in ApJL. Main text 8 pages, 4 figures. For more information of the JWST ASPIRE program please check this https URL*|
|**Abstract**| The launch of ${\it JWST}$ opens a new window for studying the connection between metal-line absorbers and galaxies at the end of the Epoch of Reionization (EoR). Previous studies have detected absorber-galaxy pairs in limited quantities through ground-based observations. To enhance our understanding of the relationship between absorbers and their host galaxies at $z>5$, we utilized the NIRCam Wide Field Slitless Spectroscopy (WFSS) to search for absorber-associated galaxies by detecting their rest-frame optical emission lines (e.g., [OIII] + H$\beta$). We report the discovery of a MgII-associated galaxy at $z=5.428$ using data from the ${\it JWST}$ ASPIRE program. The MgII absorber is detected on the spectrum of quasar J0305--3150 with a rest-frame equivalent width of 0.74$\mathring{A}$. The associated galaxy has an [OIII] luminosity of $10^{42.5}\ {\rm erg\ s^{-1}}$ with an impact parameter of 24.9 proper kiloparsecs (pkpc). The joint ${\it HST}$-${\it JWST}$ spectral energy distribution (SED) implies a stellar mass and star-formation rate of ${\rm M_* \approx 10^{8.8}}$ ${\rm M_{\odot}}$, ${\rm SFR}\approx 10\ {\rm M_{\odot}\ yr^{-1}}$. Its [OIII] equivalent width and stellar mass are typical of [OIII] emitters at this redshift. Furthermore, connecting the outflow starting time to the SED-derived stellar age, the outflow velocity of this galaxy is $\sim300\ {\rm km\ s^{-1}}$, consistent with theoretical expectations. We identified six additional [OIII] emitters with impact parameters of up to $\sim300$ pkpc at similar redshifts ($|dv|<1000\ {\rm km\ s^{-1}}$). The observed number is consistent with that in cosmological simulations. This pilot study suggests that systematically investigating the absorber-galaxy connection within the ASPIRE program will provide insights into the metal-enrichment history in the early universe. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.16926-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.16926) | **The Close AGN Reference Survey (CARS): An interplay between radio jets  and AGN radiation in the radio-quiet AGN HE 0040-1105**  |
|| M. Singha, et al. -- incl., <mark>N. Winkel</mark>, <mark>I. Smirnova-Pinchukova</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2023-10-02*|
|*Comments*| *Accepted in ApJ for publication*|
|**Abstract**| We present a case study of HE 0040-1105, an unobscured radio-quiet AGN at a high accretion rate (Eddington ratio = 0.19+/-0.04). This particular AGN hosts an ionized gas outflow with the largest spatial offset from its nucleus compared to all other AGNs in the Close AGN Reference Survey (CARS). By combining multi-wavelength observations from VLT/MUSE, HST/WFC3, VLA, and EVN we probe the ionization conditions, gas kinematics, and radio emission from host galaxy scales to the central few pc. We detect four kinematically distinct components, one of which is a spatially unresolved AGN-driven outflow located within the central 500 pc, where it locally dominates the ISM conditions. Its velocity is too low to escape the host galaxy's gravitational potential, and maybe re-accreted onto the central black hole via chaotic cold accretion. We detect compact radio emission in HE 0040-1105,within the region covered by the outflow, varying on ~20 yr timescale. We show that neither AGN coronal emission nor star formation processes wholly explain the radio morphology/spectrum. The spatial alignment between the outflowing ionized gas and the radio continuum emission on 100 pc, scales is consistent with a weak jet morphology rather than diffuse radio emission produced by AGN winds. > 90% of the outflowing ionized gas emission originates from the central 100 pc, within which the ionizing luminosity of the outflow is comparable to the mechanical power of the radio jet. Although radio jets might primarily drive the outflow in HE 0040-1105,, radiation pressure from the AGN may contribute in this process. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.16958-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.16958) | **PopSED: Population-Level Inference for Galaxy Properties from Broadband  Photometry with Neural Density Estimation**  |
|| <mark>J. Li</mark>, P. Melchior, C. Hahn, S. Huang |
|*Appeared on*| *2023-10-02*|
|*Comments*| *12 pages, 4 figures. Submitted to ApJ*|
|**Abstract**| We present PopSED, a framework for population-level inference of galaxy properties from photometric data. Unlike the traditional approach of first analyzing individual galaxies and then combining the results to determine the physical properties of the entire galaxy population, we directly make the population distribution the inference objective. We train normalizing flows to approximate the population distribution by minimizing the Wasserstein distance between the synthetic photometry of the galaxy population and the observed data. We validate our method using mock observations and apply it to galaxies from the GAMA survey. PopSED reliably recovers the redshift and stellar mass distribution of $10^{5}$ galaxies using broadband photometry within $<1$ GPU-hour, being $10^{5-6}$ times faster than the traditional SED modeling method. From the population posterior we also recover the star-forming main sequence for GAMA galaxies at $z<0.1$. With the unprecedented number of galaxies in upcoming surveys, our method offers an efficient tool for studying galaxy evolution and deriving redshift distributions for cosmological analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.17440-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.17440) | **Investigating the Drivers of Electron Temperature Variations in HII  Regions with Keck-KCWI and VLT-MUSE**  |
|| R. J. R. Vaught, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2023-10-02*|
|*Comments*| *Submitted to ApJ, 62 pages, 35 figures*|
|**Abstract**| HII region electron temperatures are a critical ingredient in metallicity determinations and recent observations reveal systematic variations in the temperatures measured using different ions. We present electron temperatures ($T_e$) measured using the optical auroral lines ([NII]$\lambda5756$, [OII]$\lambda\lambda7320,7330$, [SII]$\lambda\lambda4069,4076$, [OIII]$\lambda4363$, and [SIII]$\lambda6312$) for a sample of HII regions in seven nearby galaxies. We use observations from the Physics at High Angular resolution in Nearby Galaxies survey (PHANGS) obtained with integral field spectrographs on Keck (Keck Cosmic Web Imager; KCWI) and the Very Large Telescope (Multi-Unit Spectroscopic Explorer; MUSE). We compare the different $T_e$ measurements with HII region and interstellar medium environmental properties such as electron density, ionization parameter, molecular gas velocity dispersion, and stellar association/cluster mass and age obtained from PHANGS. We find that the temperatures from [OII] and [SII] are likely over-estimated due to the presence of electron density inhomogeneities in HII regions. We observe that differences between [NII] and [SIII] temperatures are weakly correlated with stellar association mass and molecular gas velocity dispersion. We measure high [OIII] temperatures in a subset of regions with high molecular gas velocity dispersion and low ionization parameter, which may be explained by the presence of low-velocity shocks. Of all the temperatures considered, the $T_{\rm{e}}$--$T_{\rm{e}}$ between [NII] and [SIII] temperatures have the lowest observed scatter and generally follow predictions from photoionization modeling, which suggests that these tracers reflect HII region temperatures across the various ionization zones better than [OII], [SII], and [OIII]. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.17287-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.17287) | **$Euclid$: The search for primordial features**  |
|| M. Ballardini, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2023-10-02*|
|*Comments*| **|
|**Abstract**| Primordial features, in particular oscillatory signals, imprinted in the primordial power spectrum of density perturbations represent a clear window of opportunity for detecting new physics at high-energy scales. Future spectroscopic and photometric measurements from the $Euclid$ space mission will provide unique constraints on the primordial power spectrum, thanks to the redshift coverage and high-accuracy measurement of nonlinear scales, thus allowing us to investigate deviations from the standard power-law primordial power spectrum. We consider two models with primordial undamped oscillations superimposed on the matter power spectrum, one linearly spaced in $k$-space the other logarithmically spaced in $k$-space. We forecast uncertainties applying a Fisher matrix method to spectroscopic galaxy clustering, weak lensing, photometric galaxy clustering, cross correlation between photometric probes, spectroscopic galaxy clustering bispectrum, CMB temperature and $E$-mode polarization, temperature-polarization cross correlation, and CMB weak lensing. We also study a nonlinear density reconstruction method to retrieve the oscillatory signals in the primordial power spectrum. We find the following percentage relative errors in the feature amplitude with $Euclid$ primary probes for the linear (logarithmic) feature model: 21% (22%) in the pessimistic settings and 18% (18%) in the optimistic settings at 68.3% confidence level (CL) using GC$_{\rm sp}$+WL+GC$_{\rm ph}$+XC. Combining all the sources of information explored expected from $Euclid$ in combination with future SO-like CMB experiment, we forecast ${\cal A}_{\rm lin} \simeq 0.010 \pm 0.001$ at 68.3% CL and ${\cal A}_{\rm log} \simeq 0.010 \pm 0.001$ for GC$_{\rm sp}$(PS rec + BS)+WL+GC$_{\rm ph}$+XC+SO-like both for the optimistic and pessimistic settings over the frequency range $(1,\,10^{2.1})$. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.16757.md
    + _build/html/tmp_2309.16757/./Figure_1.png
    + _build/html/tmp_2309.16757/./Figure_4.png
    + _build/html/tmp_2309.16757/./Figure_3.png
exported in  _build/html/2309.16926.md
    + _build/html/tmp_2309.16926/figures/gas_maps.png
    + _build/html/tmp_2309.16926/figures/C1.png
    + _build/html/tmp_2309.16926/figures/kin_model.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\red}{\color{red}}$
$\newcommand{\blue}{\color{blue}}$
$\newcommand{\CII}{[C \textsc{ii}]}$
$\newcommand{\Msun}{M_\odot\xspace}$
$\newcommand{\Mstar}{M_\ast\xspace}$
$\newcommand{\oi}{\mbox{\ion{O}{1}}}$
$\newcommand{\oiii}{\mbox{[\ion{O}{3}]}}$
$\newcommand{\mgii}{\ion{Mg}{2}}$
$\newcommand{\feii}{\ion{Fe}{2}}$
$\newcommand{\civ}{\ion{C}{4}}$
$\newcommand{\siiv}{\ion{Si}{4}}$
$\newcommand{\targetname}{ASPIRE-J0305M31-O3-038}$</div>



<div id="title">

# A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE): JWST Discovers an Overdensity around a Metal Absorption-selected Galaxy at $z\sim5.5$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.16757-b31b1b.svg)](https://arxiv.org/abs/2309.16757)<mark>Appeared on: 2023-10-02</mark> -  _Accepted for publication in ApJL. Main text 8 pages, 4 figures. For more information of the JWST ASPIRE program please check this https URL_

</div>
<div id="authors">

Y. Wu, et al. -- incl., <mark>Z.-L. Xie</mark>

</div>
<div id="abstract">

**Abstract:** The launch of $_ JWST_$ opens a new window for studying the connection between metal-line absorbers and galaxies at the end of the Epoch of Reionization (EoR). Previous studies have detected absorber-galaxy pairs in limited quantities through ground-based observations. To enhance our understanding of the relationship between absorbers and their host galaxies at $z>5$ , we utilized the NIRCam Wide Field Slitless Spectroscopy (WFSS) to search for absorber-associated galaxies by detecting their rest-frame optical emission lines (e.g., $\oiii$ + H $\beta$ ). We report the discovery of a $\mgii$ -associated galaxy at $z=5.428$ using data from the $_ JWST_$ ASPIRE program. The $\mgii$ absorber is detected on the spectrum of quasar J0305--3150 with a rest-frame equivalent width of 0.74 $Å$ . The associated galaxy has an $\oiii$ luminosity of $10^{42.5} {\rm erg s^{-1}}$ with an impact parameter of 24.9 proper kiloparsecs (pkpc). The joint $_ HST_$ - $_ JWST_$ spectral energy distribution (SED) implies a stellar mass and star-formation rate of ${\rm M_* \approx 10^{8.8}}$ ${\rm M_{\odot}}$ , ${\rm SFR}\approx 10 {\rm M_{\odot} yr^{-1}}$ . Its $\oiii$ equivalent width and stellar mass are typical of $\oiii$ emitters at this redshift. Furthermore, connecting the outflow starting time to the SED-derived stellar age, the outflow velocity of this galaxy is $\sim300 {\rm km s^{-1}}$ , consistent with theoretical expectations. We identified six additional $\oiii$ emitters with impact parameters of up to $\sim300$ pkpc at similar redshifts ( $|dv|<1000 {\rm km s^{-1}}$ ). The observed number is consistent with that in cosmological simulations. This pilot study suggests that systematically investigating the absorber-galaxy connection within the ASPIRE program will provide insights into the metal-enrichment history in the early universe.

</div>

<div id="div_fig1">

<img src="tmp_2309.16757/./Figure_1.png" alt="Fig2" width="100%"/>

**Figure 2. -** ** Top Left**: The JWST/NIRCam WFSS spectrum of $\targetname$. The error spectrum is shown in pink lines. ** Bottom Left**: Normalized X-SHOOTER spectrum of the quasar J0305 with $\mgii$$\lambda\lambda 2796, 2803$ and $\feii$${\rm \lambda} 2382, 2600$ absorptions at $z=5.4284$. Red-shaded regions denote the best-fit Voigt profile. Dashed blue lines indicate the redshift. ** Right:**_ JWST_/NIRCam composite RGB map of the quasar field with the pixel scale of 0.03$\arcsec$(blue: F115W, green: F200W, red: F356W). White circles denote the location of $\targetname$ and the quasar J0305. We note that the $\mgii$ absorber is in front of the quasar with the redshift of $z=5.428$. The impact parameter between the $\mgii$ absorber and $\targetname$ is $4.1$\arcsec, corresponding to 24.9 pkpc at $z=5.428$. (*spec_img*)

</div>
<div id="div_fig2">

<img src="tmp_2309.16757/./Figure_4.png" alt="Fig1" width="100%"/>

**Figure 1. -** Number excess of galaxies around $\mgii$ absorbers at $z\simeq5$, $\xi_{\rm gal-abs}(r) = \frac{1}{n_0}\frac{N}{\Delta V} - 1$.
    The red dots are the measurements obtained from our observations.
    Error bars are estimated by assuming Poissonian uncertainties in the 84\% confidence level  ([ and Gehrels 1986]()) .
    The blue-solid line indicates simulation-predicted values obtained from [Doughty and Finlator (2023)](). The observed values are consistent with that predicted from cosmological simulations.  (*Comp_with_simu*)

</div>
<div id="div_fig3">

<img src="tmp_2309.16757/./Figure_3.png" alt="Fig3" width="100%"/>

**Figure 3. -** ** Left:**$\oi$ii EWs and derived stellar masses at $z\sim6$, compared to these of local galaxies (shaded region) (MPA-JHU catalog  ([Kauffmann, Heckman and White 2003](), [Brinchmann, Charlot and White 2004]()) ). The measurement of the absorber-associated $\oi$ii emitter is shown as the red star. Grey dots indicate results obtained from the EIGER sample  ([Matthee, Mackenzie and Simcoe 2022]()) . ** Right:** SFR distribution of metal-absorber selected galaxies. Galaxies at $z>5$ are marked as dashed bars. At $z<5$, the $\civ$-associated LAEs selected from the MAGG survey  ([Galbiati, Fumagalli and Fossati 2023]())  are shown in grey. The blue bars show the $\civ$-associated LAEs at $z>5$ ([Cai, et. al 2017](), [Díaz, Ryan-Weber and Karman 2021]()) . At $z\approx6$, One ALMA-detected $\oi$-associated emitter is shown in pink  ([Wu, Cai and Neeleman 2021]()) . The $\oi$ii emitter detected with _ JWST_ is shown in red.  (*sample_comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.16757"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The Close AGN Reference Survey (CARS)$\An interplay between radio jets and AGN radiation in the radio-quiet AGN HE 0040$$-$1105

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.16926-b31b1b.svg)](https://arxiv.org/abs/2309.16926)<mark>Appeared on: 2023-10-02</mark> -  _Accepted in ApJ for publication_

</div>
<div id="authors">

M.~Singha, et al.

</div>
<div id="abstract">

**Abstract:** \textcolor{black}{We present a case study of HE 0040$-$1105, an unobscured radio-quiet AGN at a high accretion rate $\lambda_{Edd} = 0.19\pm0.04$. This particular AGN hosts an ionized gas outflow with the largest spatial offset from its nucleus compared to all other AGNs in the Close AGN Reference Survey (CARS). By combining multi-wavelength observations from VLT/MUSE, HST/WFC3, VLA, and EVN we probe the ionization conditions, gas kinematics, and radio emission from host galaxy scales to the central few pc. We detect four kinematically distinct components, one of which is a spatially unresolved AGN-driven outflow located within the central $500 \text{pc}$, where it locally dominates the ISM conditions. Its velocity is too low to escape the host galaxy's gravitational potential, and maybe re-accreted onto the central black hole via chaotic cold accretion. We detect compact radio emission in HE 0040$-$1105 within the region covered by the outflow, varying on $\sim 20$ yr timescale.We show that neither AGN coronal emission nor star formation processes wholly explain the radio morphology/spectrum. The spatial alignment between the outflowing ionized gas and the radio continuum emission on $100 \text{pc}$ scales is consistent with a weak jet morphology rather than diffuse radio emission produced by AGN winds.$> 90\%$ of the outflowing ionized gas emission originates from the central $100 \text{pc}$, within which the ionizing luminosity of the outflow is comparable to the mechanical power of the radio jet. Although radio jets might primarily drive the outflow in HE 0040$-$1105, radiation pressure from the AGN may contribute in this process.}

</div>

<div id="div_fig1">

<img src="tmp_2309.16926/figures/gas_maps.png" alt="Fig2" width="100%"/>

**Figure 2. -** Mapping HE 0040$-$1105's ionized gas flux and kinematics across the host galaxy. The left panels show the maps extracted from the single-component [$\ion${O}{3}] model, and the right panels are those for H$\alpha$. From top to bottom the panels show the emission line flux, rest-frame velocity $v$, and velocity dispersion $\sigma$, respectively. The EELR that is local to the galaxy (see Section \ref{subsection:energetics_comparison}) is highlighted by the red contour in panel (d). The contours of ionized gas outflow in the center (see Section \ref{subsec:ionized_gas_outflow}) are shown as dashed cyan lines. In addition, we highlight the contours of the kinematic features analyzed in Section \ref{subsec:resolved_host_galaxy_emission} where the red line in panel (b) describes the morphology of the receding shell C1, and the red line in panel (e) indicates the H$\alpha$ emitting receding region C3. Both [$\ion${O}{3}] emission line nebulae have similar morphologies. However, while the H$\alpha$ velocity field has a clear rotational pattern and a nucleated peak of the velocity dispersion, the [$\ion${O}{3}] shows a more chaotic motion with a velocity dispersion in C1 that is lower than average. (*fig:gas_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2309.16926/figures/C1.png" alt="Fig8" width="100%"/>

**Figure 8. -** Multi-wavelength view of the region C1. _Left panel_: MUSE optical spectra extracted from a $3 \times 3$ spaxel region (in gray), along with its multi-Gaussian component fit (in red). The green-shaded Gaussian components represent the narrow core which is a part of the EELR, whereas the dark brown-shaded components trace the red-wing component, corresponding to the region C1. _Right panel_: _HST_/WFC3 near-UV image centered at $3350 \text{Å}$. The green region describes the area covered by the deconvolved C1, and the white line denotes its centroids. A clumpy structure on the northeastern side of the nucleus is shaded with white to showcase its spatial location and morphology. C1 overlaps with the gas clumps on the north where $S/N > 5$. Such spatial coincidence indicates that C1 is a part of the clumpy region. (*fig:C1*)

</div>
<div id="div_fig3">

<img src="tmp_2309.16926/figures/kin_model.png" alt="Fig1" width="100%"/>

**Figure 1. -** Kinematic modelling of HE 0040$-$1105's stellar (left columns) and H$\alpha$ ionized gas (right columns) velocity field. From top to bottom the panels show the velocities measured from the MUSE observations (see Section \ref{subsec:stellar_velocity_field} and Section \ref{subsubsec:EELR}), the tilted-ring model fitted to it, and the map with the residual velocities normalized by the uncertainty. In both cases, the thin rotating disc provides a good description of the velocity field, although the rotation axis between both host galaxy components is tilted by $53\arcdeg$. (*fig:stellar_vel_model*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.16926"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

285  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
